# 构造微调训练数据集

借助 ChatGPT 和 GPT API 我们可以实现自动化批量构造训练数据集。

下面我们以中国古典哲学数据集为例，展示了自动构造训练集的主要流程：

- 使用 LangChain 构造训练数据样例
    - 基于 ChatGPT 设计 `System Role` 提示词
    - 使用 `OpenAI GPT-3.5-Turbo-1106` 生成基础数据
    - 解析 OpenAI GPT 生成的训练数据
    - 持久化存储`dataset.csv`训练数据集文件
    - 使用 ChatGPT 实现训练数据多样化
- 自动化批量生成训练数据集
    - 整理收集原始数据`raw_data.txt`
    - 自动解析原始数据样例 `raw_data_content[]`
    - 设计 `gen_data` 训练数据生成器函数
    - 设计训练数据生成流水线

最佳实践参考：

- 使用 GPT-3.5 生成基础数据：https://platform.openai.com/playground/p/2c7XNPgo6Y2iDxILiWfD3iPu?model=gpt-3.5-turbo-1106&mode=chat
- 使用 ChatGPT 生成数据处理代码和相关文本整理：https://chat.openai.com/share/cdfd2d1d-a75e-4cee-be49-539c010ca1b1
- GPT API 价格: https://openai.com/pricing

## 使用 OpenAI SDk 构造训练数据

In [37]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [38]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving requirements.txt to requirements (1).txt
User uploaded file "requirements (1).txt" with length 226 bytes


In [39]:
!pip install -r requirements.txt

In [40]:
pip install openai

In [42]:
from openai import OpenAI
client = OpenAI()

In [43]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {
      "role": "system",
      "content": "你是中国古典哲学大师，尤其擅长周易的哲学解读。\n\n接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。\n\n示例输入：\n\n师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。\n师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。\n\n期待结果：\n\ncontent:\"师卦\"\nsummary:\"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。\n\n师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。\n\n师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。\""
    },
    {
      "role": "user",
      "content": "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"
    },
    {
      "role": "assistant",
      "content": "content:\"蒙卦\"\nsummary:\"在周易中，蒙卦象征着教育启蒙和智慧的初期阶段，由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。坎象征水，艮象征山，代表了山下有水源，水流出山。这预示着在事物发展的起初阶段，人们会陷入蒙昧状态，因此教育成为当务之急。只有培养纯正无邪的品质，才能走上通达之路，如同泉水汇成江河。\n\n蒙卦的核心哲学是：事物发展的初期阶段必然处于蒙昧状态，但如果能把握时机，行动切合时宜，就能启蒙通达。蒙卦紧随始生卦《屯》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明物在幼稚阶段都会经历蒙昧状态，找它蒙即是稚嫩。\n\n《象》中解释说：山下出泉，蒙；君子以果行育德，意味着像泉水从山下流出一样，启蒙教育有助于培育美德。\""
    },
    {
      "role": "user",
      "content": "需卦，本卦是异卦相叠，上卦为坎，下卦为乾。乾卦象征天空辽阔，坎卦象征乌云密布。雨水之于传统农业社会而言，是最大的需求，因此用这个意象来表现。古人引申为雨将下，万物皆待，所以叫需。\n需，古体字结构是“雨上天下”，从雨而声。如苍天下雨，滋养万物及人。从人的角度看，天下雨，则不易出行耕作，因此在家喝酒吃肉、静待天时。所以，这一卦的基本象征就是等待、不进，还有饮食之义。\n \n需卦位于蒙卦之后，《序卦》之中这样解释道：“物稚不可不养也，故受之以需。需者，饮食之道也。”有所需要，也是有所等待。\n《象》曰：云上于天，需;君子以饮食宴乐。\n《象》中这段话的意思是说：需卦的卦象是乾(天)下坎(水)上，为水在天上之表象。水汽聚集天上成为云层，密云満天，但还没有下雨，需要等待;君子在这个时候需要吃喝，饮酒作乐，即在等待的时候积蓄力量。\n需卦象征等待，启示人们守正待机的道理，属于中上卦。《象》这样评断此卦：明珠土埋日久深，无光无亮到如今，忽然大风吹土去，自然显露有重新。\n"
    },
    {
      "role": "assistant",
      "content": "content:\"需卦\"\nsummary:\"在周易中，需卦是一个富有深意的卦象，由两个异卦相叠组成：下卦乾（天）和上卦坎（水）。乾象征广阔的天空，而坎象征乌云密布，暗示着迫切需要雨水的意象。古人将雨水视为万物生长的必需，因此命名为“需”。需卦的基本象征是等待和饮食之道。\n\n需卦位于蒙卦之后，《序卦》中解释为“物稚不可不养也，故受之以需。需者，饮食之道也。”说明物在幼稚状态需要得到滋养，因此受到需卦的启示。《象》中描述：云上于天，需；君子以饮食宴乐。意味着天空乌云密布，需要等待雨水，而君子则在等待的过程中积蓄力量，享受饮食和欢乐。\n\n需卦的核心哲学是：等待，启示着守正待机的道理。它属于中上卦，象征着埋藏已久的明珠，经过大风吹去尘埃后，重新显露光芒。\""
    }
  ],
  temperature=1,
  max_tokens=4095,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

## 使用 LangChain 构造训练数据

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  temperature=1,
                  max_tokens=4095,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0)

                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


In [45]:
system_content = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：

师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：

content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""


In [46]:
# 原始数据
raw_content = "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"

In [47]:
messages = [
    SystemMessage(
        content=system_content
    ),
    HumanMessage(
        content=raw_content
    ),
]

In [48]:
ai_message = chat(messages)

### 解析 OpenAI GPT 生成的训练数据

In [49]:
ai_message.content

'content:"蒙卦"\nsummary:"蒙卦是周易中一个充满智慧的卦象，由坎卦（水）和艮卦（山）两个异卦相叠而成。坎代表水，艮代表山，形成山下有水的情景。这象征着事物发展的初期阶段，充满了蒙昧和不明，需要进行教育启蒙。在这一过程中，应当培养学生纯正无邪的品质，以治蒙之道。\n\n蒙卦的核心观念是：尽管初期充满蒙昧，但抓住时机，顺时而动，可以启蒙开朗，通达治愚。因此，蒙卦象征着启蒙和通达的智慧。\n\n蒙卦位于《屯卦》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明在事物或人的幼稚阶段，就如同山下的泉水初涌，需要蒙昧阶段受到启蒙的照耀。\n\n《象》中解释蒙卦的意义是：山下有泉水涌出，这就是蒙卦的图象。君子应当果断行动，培育品德，引导启蒙之智。"'

In [50]:
text = ai_message.content
print(text)
print('++++++++++++++++++++++++++++++++++++++')

# 分割字符串来找到content和summary的位置
content_start = text.find('content:"') + len('content:"')
content_end = text.find('"', content_start)
summary_start = text.find('summary:"') + len('summary:"')
summary_end = text.rfind('"interpretation": ')

# 提取并存储content和summary
content = text[content_start:content_end].strip()
summary = text[summary_start:summary_end].strip()

print("Content:", content)
print('------------------------------------')
print("Summary:", summary)


content:"蒙卦"
summary:"蒙卦是周易中一个充满智慧的卦象，由坎卦（水）和艮卦（山）两个异卦相叠而成。坎代表水，艮代表山，形成山下有水的情景。这象征着事物发展的初期阶段，充满了蒙昧和不明，需要进行教育启蒙。在这一过程中，应当培养学生纯正无邪的品质，以治蒙之道。

蒙卦的核心观念是：尽管初期充满蒙昧，但抓住时机，顺时而动，可以启蒙开朗，通达治愚。因此，蒙卦象征着启蒙和通达的智慧。

蒙卦位于《屯卦》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明在事物或人的幼稚阶段，就如同山下的泉水初涌，需要蒙昧阶段受到启蒙的照耀。

《象》中解释蒙卦的意义是：山下有泉水涌出，这就是蒙卦的图象。君子应当果断行动，培育品德，引导启蒙之智。"
++++++++++++++++++++++++++++++++++++++
Content: 蒙卦
------------------------------------
Summary: 蒙卦是周易中一个充满智慧的卦象，由坎卦（水）和艮卦（山）两个异卦相叠而成。坎代表水，艮代表山，形成山下有水的情景。这象征着事物发展的初期阶段，充满了蒙昧和不明，需要进行教育启蒙。在这一过程中，应当培养学生纯正无邪的品质，以治蒙之道。

蒙卦的核心观念是：尽管初期充满蒙昧，但抓住时机，顺时而动，可以启蒙开朗，通达治愚。因此，蒙卦象征着启蒙和通达的智慧。

蒙卦位于《屯卦》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明在事物或人的幼稚阶段，就如同山下的泉水初涌，需要蒙昧阶段受到启蒙的照耀。

《象》中解释蒙卦的意义是：山下有泉水涌出，这就是蒙卦的图象。君子应当果断行动，培育品德，引导启蒙之智。


### 持久化存储训练数据集文件

In [51]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

# 新建CSV文件并写入数据
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入标题行
    writer.writerow(['content', 'summary'])
    # 写入数据行
    writer.writerow([content, summary])


### 数据增强：构造多样化的提问方式

In [52]:
def generate_question_summary_pairs(content, summary):
    """
    生成40对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含40对提问和总结的列表。
    """
    # 40种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "{}和教育启蒙有什么联系？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}在周易中怎样表达教育的概念？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "{}和启蒙教育之间有何联系？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]
    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions]
    return question_summary_pairs

In [53]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
print(f'content: {content}')
print(f'summary: {summary}')
pairs = generate_question_summary_pairs(content, summary)

# 将结果写入CSV文件
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['content', 'summary'])
    for pair in pairs:
        writer.writerow(pair)


content: 蒙卦
summary: 蒙卦是周易中一个充满智慧的卦象，由坎卦（水）和艮卦（山）两个异卦相叠而成。坎代表水，艮代表山，形成山下有水的情景。这象征着事物发展的初期阶段，充满了蒙昧和不明，需要进行教育启蒙。在这一过程中，应当培养学生纯正无邪的品质，以治蒙之道。

蒙卦的核心观念是：尽管初期充满蒙昧，但抓住时机，顺时而动，可以启蒙开朗，通达治愚。因此，蒙卦象征着启蒙和通达的智慧。

蒙卦位于《屯卦》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明在事物或人的幼稚阶段，就如同山下的泉水初涌，需要蒙昧阶段受到启蒙的照耀。

《象》中解释蒙卦的意义是：山下有泉水涌出，这就是蒙卦的图象。君子应当果断行动，培育品德，引导启蒙之智。


## 自动化批量生成训练数据流水线

原始数据来源：https://www.zhouyi.cc/zhouyi/yijing64/4103.html

In [54]:
# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('/content/drive/MyDrive/data/chatglm/raw_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)

In [55]:
# 输出结果以验证
for i, sample in enumerate(raw_content_data[:5]):  # 打印前5个样例以检查
    print(f"样例 {i+1}:")
    print(sample)
    print("------")


样例 1:
《蒙卦》
蒙卦原文
蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。
象曰：山下出泉，蒙。君子以果行育德。
白话文解释
蒙卦：通泰。不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。第一次占筮，神灵告诉了他。轻慢不敬的再三占筮，轻慢不敬的占筮，神灵就不会告诉他。但还是吉利的卜问。
《象辞》说：上卦为艮，象征山；下卦为坎，象征泉。山下有泉，泉水喷涌而出，这是蒙卦的卦象。君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。
《断易天机》解
蒙卦艮上坎下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。
北宋易学家邵雍解
智慧未开，蒙昧闭塞；犹豫不决，缺乏果断。
得此卦者，智慧犹如童蒙，不辨是非，迷失方向；若能顺贤师良友之教，启其聪明则亨通。
台湾国学大儒傅佩荣解
时运：蓄积德行，出而用世。
财运：矿山生意，果决则吉。
家宅：君子居吉；婚姻之始。
身体：驱去邪热，可保平安。
传统解卦
这个卦是异卦（下坎上艮）相叠，艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜，因此，具有启蒙和通达的卦象。
大象：蒙者，昏而无所见也，故宜「启蒙」。
运势：初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。
事业：事业开始，混乱无序，危机四伏，以勇敢坚毅的行动可以扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。
经商：务必小心谨慎，不得急功近利，尤其应树立高尚的商业道德，以良好的信誉提高竞争力而取胜。
求名：必须接受良好的基础教育，陶冶情操。且动机纯正，可以达到目的。
婚恋：注意考察对方品德，不可以金钱为诱铒。夫妻需相互宽容、理解。
决策：有时会陷入迷惘困顿的境地，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。
------
样例 2:
《屯卦》
屯卦原文
屯。元，亨，利，贞。勿用，有攸往，利建侯。
象曰：云，雷，屯；君子以经纶。
白话文解释
屯卦。大吉大利，吉利的占卜。不利于出门。有利于建国封侯。
《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩

### 将以上的所有模块，整合到一起，自动化生成数据

In [56]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# 初始化LangChain的GPT-3.5调用
chat = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  temperature=1,
                  max_tokens=4095,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0)

def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


In [57]:
# 示例调用（使用 raw_data.txt 中解析的数据样例）
generated_content = gen_data(raw_content_data[12])
print(generated_content)

content:"谦卦"
summary:"谦卦"是周易中的一卦，由两个异卦相叠而成：下卦艮（山）上卦坤（地）。这一卦象通泰，预示着君子将有所成就。在卦象中，地下有山，内高外低，寓意谦逊和裒多益寡，称物平施。谦者，谦虚、逊让之意，代表内心的谦虚和外在行为的谦恭。谦卦提倡裁取多余以增益缺乏，衡量财物多寡来公平施予。

谦卦为兑宫五世卦，解为谦逊、谦虚，内卦艮和外卦坤都是谦虚之象，故为吉利之卦。古代经学家解释道，谦卦象征着谦和忍让，利用谦虚可使诸事顺利。得此卦者，预示着吉利平安，前途光明，而骄横者则必败。台湾国学大儒傅佩荣指出，谦卦运势平顺，财运合理保障，家宅平安，身体内郁之症可通过放宽心胸而得以缓解。

传统解卦认为，谦卦代表内高外低的谦虚之德。运势吉利平安，但需要去除骄横之气。在事业上，谦卦提示尚未被人器重，但有望为人所发现，需要保持谦虚的品德，埋头苦干。在经商方面，谦卦提示较为顺利，但需小心谨慎。在求名方面，天资聪明者应保持谦逊好学的精神。在婚恋方面，谦卦提示需要双方共同努力维系美满的婚姻。决策上，谦卦暗示着内在的谦虚美德、团结他人的能力、对他人长处的吸取，以及遇到困难能化险为夷的特点。同时，也需要特别注意学业、德业和修养。"


In [58]:
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content，提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 分割字符串来找到content和summary的位置
    # content_start = ai_message_content.find('content:"') + len('content:"')
    # content_end = ai_message_content.find('",\n  "summary": ')
    # summary_start = ai_message_content.find('summary:"') + len('summary:"')
    # summary_end = ai_message_content.rfind('",\n  "interpretation": ')

    text = ai_message_content
    content_start = text.find('content:"') + len('content:"')
    content_end = text.find('"', content_start)
    summary_start = text.find('summary:"') + len('summary:"')
    summary_end = text.rfind('"interpretation": ')
    # 提取并存储content和summary
    content = ai_message_content[content_start:content_end].strip()
    summary = ai_message_content[summary_start:summary_end].strip()

    return content, summary


In [59]:
# 示例调用（使用假设的gen_data函数返回的文本）
content, summary = dataset_parser(generated_content)
print("Content:", content)
print("Summary:", summary)

Content: 谦卦
Summary: 谦卦"是周易中的一卦，由两个异卦相叠而成：下卦艮（山）上卦坤（地）。这一卦象通泰，预示着君子将有所成就。在卦象中，地下有山，内高外低，寓意谦逊和裒多益寡，称物平施。谦者，谦虚、逊让之意，代表内心的谦虚和外在行为的谦恭。谦卦提倡裁取多余以增益缺乏，衡量财物多寡来公平施予。

谦卦为兑宫五世卦，解为谦逊、谦虚，内卦艮和外卦坤都是谦虚之象，故为吉利之卦。古代经学家解释道，谦卦象征着谦和忍让，利用谦虚可使诸事顺利。得此卦者，预示着吉利平安，前途光明，而骄横者则必败。台湾国学大儒傅佩荣指出，谦卦运势平顺，财运合理保障，家宅平安，身体内郁之症可通过放宽心胸而得以缓解。

传统解卦认为，谦卦代表内高外低的谦虚之德。运势吉利平安，但需要去除骄横之气。在事业上，谦卦提示尚未被人器重，但有望为人所发现，需要保持谦虚的品德，埋头苦干。在经商方面，谦卦提示较为顺利，但需小心谨慎。在求名方面，天资聪明者应保持谦逊好学的精神。在婚恋方面，谦卦提示需要双方共同努力维系美满的婚姻。决策上，谦卦暗示着内在的谦虚美德、团结他人的能力、对他人长处的吸取，以及遇到困难能化险为夷的特点。同时，也需要特别注意学业、德业和修养。


In [60]:
import csv
import datetime
import os
import time

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('/content/drive/MyDrive/data/chatglm/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"/content/drive/MyDrive/data/chatglm/zhouyi_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = dataset_parser(ai_message_content)

            print("II Content:", content)
            print("II Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)

            time.sleep(15)

In [62]:
# 执行主函数
main()

II Content: {
II Summary: {
  "content": "蒙卦",
  "summary": "蒙卦是一个通泰的卦象。它代表的是不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。第一次占筮即得神灵之告示，但若轻慢不敬地再三占筮，神灵将不再告知。但是，仍然是一个吉利的卜问。按照《象辞》，此卦上卦为艮，象征山；下卦为坎，象征泉。山下有泉，泉水喷涌而出，这是蒙卦的卦象。君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。",
II Content: 屯卦
II Summary: 在周易中，屯卦是一个具有深远哲理的卦象。它由两个异卦相叠组成：下卦震（雷）和上卦坎（水）。这一卦象预示着大吉大利，是一种吉利的占卜。然而，却不利于出门，却有利于建国封侯。

据《象辞》解释，屯卦的卦象为坎上震下，坎为云，震为雷。预示着云行于上，雷动于下，因此君子观此卦象，可以取法于云雷之恩泽和威严，来治理国家事务。

现代解释认为，屯卦代表着困难与动荡，需要坚毅果敢方能获得吉祥。因此，时势对于身处困境的人而言，需要守而不进，多加辛苦努力排除困难。同时，财运上创业维艰，婚姻方面初婚不和，对于家宅则需要修缮住宅。

传统解卦中，屯卦象征着万物始生，充满了艰难险阻，但顺时应运就可以欣欣向荣。这一阶段的运势是身处困境，需要步步为营，但却有初难后解之意。在事业上，要勇往直前，灵活机动，务必要有他人相助。

总的来说，屯卦预示着初始困难，需要坚忍不拔的毅力和锲而不舍的奋斗精神，却常常因为不为他人理解而陷入孤独苦闷。但如果能以乐观主义精神处世，最终将收获成功。"
II Content: 需卦
II Summary: 在周易中，需卦描述了抓到俘虏的情景。这是一个大吉大利，吉利的卜问，暗示着有利于涉水渡河。需卦由上卦坎（云）和下卦乾（天）组成，预示着云浮聚于天上，待时降雨。因此，君子观此卦象，可以宴饮安乐，待时而动。

需卦的核心哲学是：踌躇和期待。尽管面临着险阻，但应当等待并细心观察，涉大川则会获利。这是一个需要耐心等待和审时度势的卦象。

得到需卦的占卜者，应当明白时机尚未成熟，需要耐心等待，不可急进，以免见凶。智者必须待时而行，观时待变。对于事业、经商、求名、婚恋等方面，都需要审时度势，耐心等待，并坚守中正，化险为夷，最终必将成功。"

content:"《象

### 异常分析


训练第一个 epoch 时，Training Loss 比较奇怪：

```
Step	Training Loss
1	3.594100
2	4.049100
3	3.091200
4	3.381700
5	3.547800
6	2.610200
7	2.657900
8	3.163900
```

通过解析 GPT-3.5-Turbo-1106 生成结果发现问题

In [ ]:
def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:"{卦名}"
        summary:"{内容}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [ ]:
# 执行主函数
main()

Content: 蒙卦
Summary: 在周易中，蒙卦代表着通泰。它由艮卦（山）上卦坎卦（水）组成，暗示山下有泉的景象。这个卦象意味着初次卜筮即可获得神灵指示，但如果持续轻慢不敬地卜筮，神灵将不再指示。尽管如此，卜问仍然是吉利的。上卦艮象征着山，下卦坎象征着泉水，山下有泉喷涌，寓意蒙卦。君子观此卦象，取法于一往无前的山泉，以果敢坚毅的行动来培养自身的品德。

蒙卦的运势初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。在事业开始阶段，需要勇敢坚毅的行动以扭转局面，同时接受严格教育，脚踏实地，避免好高骛远。在经商过程中，务必小心谨慎，树立高尚的商业道德。在求名过程中，需要接受良好的基础教育，陶冶情操。婚恋方面则需要注意对方品德，夫妻需相互宽容、理解。在决策中，如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。
Content: 屯卦
Summary: 在周易中，屯卦是一个很有深意的卦象，它由下卦震（雷）和上卦坎（水）相叠组成。这一卦象预示着困难和危险，需要坚毅果敢才能化解，并且隐含着在劳苦之后必有美好结果的含义。

屯卦的核心哲学是：在万物始生之时，困难和艰难必将出现，但只有经过辛苦努力和坚守，才能克服困难，通达于事。此卦提醒人们须守时而行，须多加辛苦努力，排除困难，方能化解险境，通达美好。

在运势上，此卦意味着身处困境，需要步步为营，但随之而来的是初难后解的转机。在事业上，则需要勇往直前，灵活机动，克服困难才能获得成功，但也需要他人相助。在求名、婚恋等方面，同样需要坚忍不拔的毅力和勇气，但最终将获得成功。

总之，屯卦在周易中是一个充满着困难与挑战，但同时也带来着希望和转机的卦象。
Content: 需卦
Summary: 在周易中，需卦代表着捕捉俘虏的意涵。这是一个大吉大利的卦象，吉利的占卜问题有利于涉水渡河。

需卦的上卦为坎，象征着云；下卦为乾，代表天。云浮聚于天上，待时降雨，这正是需卦的卦象所要表达的。君子观此卦象，可以放心享受饮食宴乐，待时而动。

从《断易天机》来看，需卦的核心是踌躇和期待，虽然内心坚强，但前方有险阻，因此需要等待，涉大川则会获利。

关于时运、财运、家宅和身体等方面的解释也都强调时机尚未成熟，需要耐心等待，并且不要冒险。而在传统解卦中，也强调了宜稳健之妥，不可冒失行动，一定要观时待变，耐心